In [5]:
import numpy as np
import os
import pandas as pd
import time
from gurobipy import Model,GRB,LinExpr,quicksum
import random
import pickle
import matplotlib.pyplot as plt

cwd = os.getcwd()
plt.close('all')

# Set random seed
random.seed(42)

##################################
### Define initial set of bins ###
##################################

toy_model = True

if toy_model is False:

    # Dimensions of bins
    K   = {0:[150,100],
           1:[180,120]}
    
    # Costs of bins
    C_k = [100,130]
    
    ###########################
    ### Define set of items ###
    ###########################
    N = 15
    
    W_bin_max   = np.max(np.array(list(K.values()))[:,0])
    H_bin_max   = np.max(np.array(list(K.values()))[:,1])
    
    W_min = int(0.15*W_bin_max) 
    W_max = int(0.8*W_bin_max)
    H_min = int(0.15*H_bin_max) 
    H_max = int(0.8*H_bin_max)
    
    R = {i:[random.randint(W_min, W_max),random.randint(H_min, H_max)] for i in range(0,N)}
    
else:
    N   = 9
    K   = {0:[15,12]}
    C_k = [1]
    
    W_bin_max   = np.max(np.array(list(K.values()))[:,0])
    H_bin_max   = np.max(np.array(list(K.values()))[:,1])
    
    W_min = int(0.15*W_bin_max) 
    W_max = int(0.8*W_bin_max)
    H_min = int(0.15*H_bin_max) 
    H_max = int(0.8*H_bin_max)
    
    R = {0:[5,3],
         1:[3,3],
         2:[5,4],
         3:[2,9],
         4:[4,9],
         5:[6,4],
         6:[5,5],
         7:[6,3],
         8:[5,3]}
    
    

############################################################
### Define "inflated" set of bins given the set of items ###
############################################################

K_final = {}
for idx_b,k in enumerate(list(K.keys())):
    for idx_i  in R.keys():
        K_final[idx_b*N+idx_i] = [idx_b,K[idx_b],C_k[idx_b]]

K_final

{0: [0, [15, 12], 1],
 1: [0, [15, 12], 1],
 2: [0, [15, 12], 1],
 3: [0, [15, 12], 1],
 4: [0, [15, 12], 1],
 5: [0, [15, 12], 1],
 6: [0, [15, 12], 1],
 7: [0, [15, 12], 1],
 8: [0, [15, 12], 1]}

In [3]:
#################################
### 2D BPP OPTIMIZATION MODEL ###
#################################

print('Setting up model')

start_time = time.time()  

# Setup model
model = Model()

print('Creating decision variables')
l = {}
b = {}
f = {}
z = {}
x = {}
y = {}
r = {}

for i in R.keys():
    for j in R.keys():
        if j != i:
            l[i,j]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="l[%s,%s]"%(i,j))
            b[i,j]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="b[%s,%s]"%(i,j))

for i in R.keys():
    for k in K_final.keys():
        f[i,k]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="f[%s,%s]"%(i,k))
        

for k in K_final.keys():
    z[k]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="z[%s]"%(k))
    
for i in R.keys():
    x[i]=model.addVar(lb=0, ub=W_bin_max, vtype=GRB.CONTINUOUS,name="x[%s]"%(i))
    y[i]=model.addVar(lb=0, ub=H_bin_max, vtype=GRB.CONTINUOUS,name="y[%s]"%(i))
    
for i in R.keys():
    for aa in range(0,2):
        for bb in range (0,2):
            r[i,aa,bb]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="r[%s,%s,%s]"%(i,aa,bb))

model.update()



Setting up model
Creating decision variables
